# Imports

In [75]:
import requests

import pandas as pd
import numpy as np

from collections import Counter
import string
import random

import requests as r

from transformers import BartTokenizer, BartModel, pipeline # base model

# Data Set Up

In [12]:
print(requests.get('https://api.openalex.org/works?filter=concepts.id:C119857082').json()['meta']['count']) # Machine Learning
print(requests.get('https://api.openalex.org/works?filter=concepts.id:C178790620').json()['meta']['count']) # Organic Chemistry
print(requests.get('https://api.openalex.org/works?filter=concepts.id:C153911025').json()['meta']['count']) # Molecular Biology
print(requests.get('https://api.openalex.org/works?filter=concepts.id:C109214941').json()['meta']['count']) # Particle Physics
print(requests.get('https://api.openalex.org/works?filter=concepts.id:C162324750').json()['meta']['count']) # Economics


3172376
14043527
2267323
670109
16311016


In [49]:
list = [1,2,3,4,5,6]
 
# initializing the value of n
n = 2
 
# printing n elements from list
x = (random.sample(list, n))
x

[2, 1]

In [71]:
ml_papers = random.sample(requests.get('https://api.openalex.org/works?filter=concepts.id:C119857082').json()['results'], 25)
ochem_papers = random.sample(requests.get('https://api.openalex.org/works?filter=concepts.id:C178790620').json()['results'], 25)
bio_papers = random.sample(requests.get('https://api.openalex.org/works?filter=concepts.id:C153911025').json()['results'], 25)
phy_papers = random.sample(requests.get('https://api.openalex.org/works?filter=concepts.id:C109214941').json()['results'], 25)
econ_papers = random.sample(requests.get('https://api.openalex.org/works?filter=concepts.id:C162324750').json()['results'], 25)

In [74]:
ml_papers[0]['doi']

'https://doi.org/10.1126/science.220.4598.671'

# General Testing

Single Abstract

In [76]:
base_summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

Downloading: 100%|██████████| 1.58k/1.58k [00:00<00:00, 1.59MB/s]


OSError: Windows requires Developer Mode to be activated, or to run Python as an administrator, in order to create symlinks.
In order to activate Developer Mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development

In [ ]:
import os, warnings

import torch
from transformers import *
from transformers.utils import logging as hf_logging
from fastai.text.all import *

from blurr.text.data.all import *
from blurr.text.modeling.all import *

path = untar_data('D:/Scilnk/TestSciEssenceData')
model_path = Path('bart-base')
imdb_df = pd.read_csv(path / 'texts.csv')

model_path = Path("models")
imdb_df = pd.read_csv(path / "texts.csv")

n_labels = len(imdb_df["label"].unique())

pretrained_model_name = "bart-base"

config = AutoConfig.from_pretrained(pretrained_model_name)
config.num_labels = n_labels

hf_arch, hf_config, hf_tokenizer, hf_model = get_hf_objects(
    pretrained_model_name,
    model_cls=model_cls, 
    config=config
)

In [ ]:
blocks = (
    TextBlock(hf_arch, hf_config, hf_tokenizer, hf_model), 
    CategoryBlock
)
dblock = DataBlock(
    blocks=blocks, 
    get_x=ColReader("text"), 
    get_y=ColReader("label"), 
    splitter=ColSplitter()
)

dls = dblock.dataloaders(imdb_df, bs=4)

dls.show_batch(dataloaders=dls, max_n=2, trunc_at=250)

model = SummarizerWraper(hf_model)

learn = Learner(
    dls,
    hf_model,
    opt_func=partial(Adam, decouple_wd=True),
    loss_func=CrossEntropyLossFlat(),
    metrics=[accuracy],
    cbs=[BaseModelCallback],
    splitter=blurr_splitter,
)

learn.freeze()

learn.fit_one_cycle(3, lr_max=1e-3)